# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/antoine/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace InputLayer {
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };
	
			return lFeatures;
		}
	
		std::vector<std::string> get_output_names(){
			std::vector<std::string> lOutputs = { "input_0", "input_1", "input_2", "input_3" };
	
			return lOutputs;
		}
	
		tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
	
			tTable lTable;
	
			lTable["input_0"] = { Feature_0 };
			lTable["input_1"] = { Feature_1 };
			lTable["input_2"] = { Feature_2 };
			lTable["input_3"] = { Feature_3 };
	
			return lTable;
		}
	
		tTable compute_model_outputs_from_table( tTable const & iTable) {
			tTable lTable = compute_features(iTable.at("Feature_0")[0], iTable.at("Feature_1")[0], iTable.at("Feature_2")[0], iTable.at("Feature_3")[0]);
	
			return lTable;
		}
	
	} // eof namespace InputLayer
	

	namespace HiddenLayer_1 {
	
		std::vector<std

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139917194487648.exe /tmp/sklearn2sql_cpp_139917194487648.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.96156579129705,0.03778076728703,0.00065344141591,-0.03919229060827,-3.27595510777057,-7.33325767556865,0,0.96156579129705
1,,,,0.93173855594866,0.06664172008047,0.00161972397087,-0.07070302301785,-2.70842446992503,-6.42549953260882,0,0.93173855594866
2,,,,0.94979683249690,0.04907162900131,0.00113153850179,-0.05150717778855,-3.01447423193166,-6.78417706626115,0,0.94979683249690
3,,,,0.92431463449107,0.07357132113168,0.00211404437725,-0.07870275178355,-2.60949998772091,-6.15915239967622,0,0.92431463449107
4,,,,0.96398961541167,0.03538860712405,0.00062177746428,-0.03667475682415,-3.34136534329551,-7.38292830371533,0,0.96398961541167
5,,,,0.95499533780726,0.04418603557404,0.00081862661870,-0.04604882039054,-3.11934647705608,-7.1078824770

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.001281,0.144817,0.853902,-6.659898,-1.932283,-0.157939,2,0.853902
74,74,NaN,NaN,NaN,0.044145,0.719450,0.236406,-3.120286,-0.329269,-1.442206,1,0.719450
9,9,NaN,NaN,NaN,0.937012,0.061636,0.001352,-0.065059,-2.786511,-6.606011,0,0.937012
88,88,NaN,NaN,NaN,0.051580,0.663153,0.285267,-2.964613,-0.410749,-1.254331,1,0.663153
25,25,NaN,NaN,NaN,0.911936,0.085823,0.002241,-0.092186,-2.455471,-6.100618,0,0.911936
5,5,NaN,NaN,NaN,0.954995,0.044186,0.000819,-0.046049,-3.119346,-7.107882,0,0.954995
48,48,NaN,NaN,NaN,0.965995,0.033502,0.000503,-0.034597,-3.396158,-7.594541,0,0.965995
117,117,NaN,NaN,NaN,0.001243,0.264321,0.734436,-6.690369,-1.330590,-0.308653,2,0.734436
83,83,NaN,NaN,NaN,0.006383,0.371188,0.622429,-5.054107,-0.991046,-0.474126,2,0.622429
105,105,NaN,NaN,NaN,0.000629,0.191160,0.808211,-7.370635,-1.654645,-0.212933,2,0.808211


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.001281,0.144817,0.853902,-6.659898,-1.932283,-0.157939,2
74,74,NaN,NaN,NaN,0.044145,0.719450,0.236406,-3.120286,-0.329269,-1.442206,1
9,9,NaN,NaN,NaN,0.937012,0.061636,0.001352,-0.065059,-2.786511,-6.606011,0
88,88,NaN,NaN,NaN,0.051580,0.663153,0.285267,-2.964613,-0.410749,-1.254331,1
25,25,NaN,NaN,NaN,0.911936,0.085823,0.002241,-0.092186,-2.455471,-6.100618,0
5,5,NaN,NaN,NaN,0.954995,0.044186,0.000819,-0.046049,-3.119346,-7.107882,0
48,48,NaN,NaN,NaN,0.965995,0.033502,0.000503,-0.034597,-3.396158,-7.594541,0
117,117,NaN,NaN,NaN,0.001243,0.264321,0.734436,-6.690369,-1.330590,-0.308653,2
83,83,NaN,NaN,NaN,0.006383,0.371188,0.622429,-5.054107,-0.991046,-0.474126,2
105,105,NaN,NaN,NaN,0.000629,0.191160,0.808211,-7.370635,-1.654645,-0.212933,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.001281,0.144817,0.853902,-6.659898,-1.932283,-0.157939,...,NaN,NaN,0.001281,0.144817,0.853902,-6.659898,-1.932283,-0.157939,2,0.853902
74,74,NaN,NaN,NaN,0.044145,0.719450,0.236406,-3.120286,-0.329269,-1.442206,...,NaN,NaN,0.044145,0.719450,0.236406,-3.120286,-0.329269,-1.442206,1,0.719450
9,9,NaN,NaN,NaN,0.937012,0.061636,0.001352,-0.065059,-2.786511,-6.606011,...,NaN,NaN,0.937012,0.061636,0.001352,-0.065059,-2.786511,-6.606011,0,0.937012
88,88,NaN,NaN,NaN,0.051580,0.663153,0.285267,-2.964613,-0.410749,-1.254331,...,NaN,NaN,0.051580,0.663153,0.285267,-2.964613,-0.410749,-1.254331,1,0.663153
25,25,NaN,NaN,NaN,0.911936,0.085823,0.002241,-0.092186,-2.455471,-6.100618,...,NaN,NaN,0.911936,0.085823,0.002241,-0.092186,-2.455471,-6.100618,0,0.911936
5,5,NaN,NaN,NaN,0.954995,0.044186,0.000819,-0.046049,-3.119346,-7.107882,...,NaN,NaN,0.954995,0.044186,0.000819,-0.046049,-3.119346,-7.107882,0,0.954995
48,48,NaN,NaN,NaN,0.965995,0.033502,0.000503,-0.034597,-3.396158,-7.594541,...,NaN,NaN,0.965995,0.033502,0.000503,-0.034597,-3.396158,-7.594541,0,0.965995
117,117,NaN,NaN,NaN,0.001243,0.264321,0.734436,-6.690369,-1.330590,-0.308653,...,NaN,NaN,0.001243,0.264321,0.734436,-6.690369,-1.330590,-0.308653,2,0.734436
83,83,NaN,NaN,NaN,0.006383,0.371188,0.622429,-5.054107,-0.991046,-0.474126,...,NaN,NaN,0.006383,0.371188,0.622429,-5.054107,-0.991046,-0.474126,2,0.622429
105,105,NaN,NaN,NaN,0.000629,0.191160,0.808211,-7.370635,-1.654645,-0.212933,...,NaN,NaN,0.000629,0.191160,0.808211,-7.370635,-1.654645,-0.212933,2,0.808211


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
